In [1]:
import pandas as pd
import json
import os
import numpy as np
import difflib
import re
import sys

from tqdm import tqdm
from collections import deque
from IPython.display import display
from IPython.core.debugger import set_trace
from datetime import date

sys.path.append('../..')

In [2]:
DATA = './source/wakatime.json'
SCHEMA = 'wakatime'

with open(DATA, 'r') as f:
    data = json.load(f)

In [3]:
deques = {}

In [4]:
for day in tqdm(data['days']):
    date = day['date']
    for project in day['projects']:
        name = project['name']
        for key, date_data in project.items():
            if key == 'name':
                continue
            try:
                data_deque = deques[key]
            except KeyError:
                data_deque = deque()
                deques[key] = data_deque
            if key == 'grand_total':
                data_deque.append({"date": date, "project": name, **date_data})
            else:
                for datum in date_data:
                    data_deque.append({"date": date, "project": name, **datum})

100%|██████████| 582/582 [00:00<00:00, 12569.82it/s]


In [5]:
dfs = {name:pd.DataFrame(data) for name, data in deques.items()}
for name, df in dfs.items():
    df['total_minutes'] = df['total_seconds'] / 60
    df['date'] = pd.to_datetime(df['date'])
    # df['date'] = df['date'].apply(lambda dt: dt.date())
    df = df.drop(['total_seconds'], axis=1)
    dfs[name] = df
    display(f"{name}: {len(df)}")
    display(df.head())

'branches: 2257'

,date,project,digital,hours,minutes,name,percent,seconds,text,total_minutes
0,2018-11-25,mse_teacher_plan,1:57:24,1,57,master,100.0,24,1 hr 57 mins,117.407750
1,2018-11-26,web_curswork,4:28:51,4,28,master,100.0,51,4 hrs 28 mins,268.860583
2,2018-11-26,web_coursework,3:15:36,3,15,master,100.0,36,3 hrs 15 mins,195.613400
3,2018-11-26,mse_teacher_plan,1:03:37,1,3,master,100.0,37,1 hr 3 mins,63.622117
4,2018-11-27,web_curswork,8:13:50,8,13,master,100.0,50,8 hrs 13 mins,493.839200


'categories: 2247'

,date,project,digital,hours,minutes,name,percent,seconds,text,total_minutes
0,2018-11-25,mse_teacher_plan,1:57:24,1,57,Coding,100.0,24,1 hr 57 mins,117.407750
1,2018-11-26,web_curswork,4:28:51,4,28,Coding,100.0,51,4 hrs 28 mins,268.860583
2,2018-11-26,web_coursework,3:15:36,3,15,Coding,100.0,36,3 hrs 15 mins,195.613400
3,2018-11-26,mse_teacher_plan,1:03:37,1,3,Coding,100.0,37,1 hr 3 mins,63.622117
4,2018-11-27,web_curswork,8:13:50,8,13,Coding,100.0,50,8 hrs 13 mins,493.839200


'dependencies: 22247'

...

,date,project,digital,hours,minutes,name,percent,seconds,text,total_minutes
0,2018-11-25,mse_teacher_plan,1:14:31,1,14,HTML,63.47,31,1 hr 14 mins,74.517583
1,2018-11-25,mse_teacher_plan,0:42:43,0,42,JavaScript,36.39,43,42 mins,42.729100
2,2018-11-25,mse_teacher_plan,0:00:09,0,0,Python,0.14,9,9 secs,0.161067
3,2018-11-26,web_curswork,3:21:12,3,21,TypeScript,74.84,12,3 hrs 21 mins,201.214317
4,2018-11-26,web_curswork,0:51:05,0,51,Properties,19.01,5,51 mins,51.099400


'machines: 2928'

,date,project,digital,hours,machine_name_id,minutes,name,percent,seconds,text,total_minutes
0,2018-11-25,mse_teacher_plan,1:57:24,1,3acd40ab-b3f9-43ce-a1b5-601b5a3bd115,57,pavel-MS-7B38 from 95.221.69.223,100.00,24,1 hr 57 mins,1.174078e+02
1,2018-11-26,web_curswork,4:28:51,4,3acd40ab-b3f9-43ce-a1b5-601b5a3bd115,28,pavel-MS-7B38 from 95.221.69.223,100.00,51,4 hrs 28 mins,2.688606e+02
2,2018-11-26,web_coursework,3:09:26,3,1be7b9af-d39b-417f-8603-312cfc70d6a3,9,DESKTOP-LHTP4MS from 176.59.17.193,96.85,26,3 hrs 9 mins,1.894424e+02
3,2018-11-26,web_coursework,0:06:10,0,ed1d38c0-a564-4f1e-b01d-bfe7b4082c6a,6,DESKTOP-LHTP4MS from 176.59.12.64,3.15,10,6 mins,6.171033e+00
4,2018-11-26,web_coursework,0:00:00,0,4cfa62f6-a387-4422-b5ac-4f666f8ca199,0,DESKTOP-LHTP4MS from 95.221.69.223,0.00,0,0 secs,1.589457e-08


'operating_systems: 1773'

,date,project,digital,hours,minutes,name,percent,seconds,text,total_minutes
0,2018-11-25,mse_teacher_plan,1:57:24,1,57,Linux,100.0,24,1 hr 57 mins,117.407750
1,2018-11-26,web_curswork,4:28:51,4,28,Linux,100.0,51,4 hrs 28 mins,268.860583
2,2018-11-26,web_coursework,3:15:36,3,15,Windows,100.0,36,3 hrs 15 mins,195.613400
3,2018-11-26,mse_teacher_plan,1:03:37,1,3,Linux,100.0,37,1 hr 3 mins,63.622117
4,2018-11-27,web_curswork,8:13:50,8,13,Linux,100.0,50,8 hrs 13 mins,493.839200


In [6]:
from api import DBConn

DBConn()

DBConn.engine.execute(f'CREATE SCHEMA IF NOT EXISTS {SCHEMA}')

for name, df in tqdm(dfs.items()):
    df.to_sql(name, schema=SCHEMA, con=DBConn.engine, if_exists='replace')

100%|██████████| 9/9 [00:14<00:00,  1.63s/it]
